In [1]:
# Import dependencies.

import json
import pandas as pd
import numpy as np
import sklearn as skl
import numpy
import datetime
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# From dependency imports
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score,r2_score
from sklearn.linear_model import LogisticRegression
from sqlalchemy import create_engine
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dcohen\.conda\envs\PythonData\lib\site-packages\tensorf

In [2]:
### Configure settings for Relational Database Service, and defining database info in config variable.
jdbc_url="jdbc:postgresql://34.67.52.115/team5k"
config = {'user': 'postgres', 
          "password": "team5kteam5k", 
          "driver":"org.postgresql.Driver",
          "location": "34.67.52.115",
          "db": "team5k",
          "port": "5432"}

In [3]:
### Flag to print Informational message
dmdebug=True

### Creating variable holding postgres info in format "postgres://[user]:[password]@[location]:[port]/[database]".
create_engine_str = ('postgresql://' 
                     + config["user"] 
                     + ":" + config["password"] 
                     + "@" + config["location"] 
                     + ":" + config["port"] 
                     + "/" + config["db"])

In [4]:
# Connecting to Postgres instance.
engine = create_engine(create_engine_str)

In [5]:
# Printing info for table names. 
print (engine.table_names())

['education', 'committee_summary_2020', 'donations', 'agg_county_votes', 'agg_county_donors', 'fec_donor_az', 'health_metrics', 'birth_death_rate', 'postal_codes', 'fec_donor_mi', 'fec_donor_wi', 'fec_committee', 'fec_donor_pa', 'pres_votes_6t', 'unemployment', 'fec_donor_nc', 'fec_donor_fl']


In [6]:
### Get voter data
votes_df = pd.read_sql_query('select * from "agg_county_votes"',con=engine)
votes_df

,index,blue_votes,red_votes,other_votes,total_votes,percent_blue,percent_red,percent_other,county,state,election_year,PopPct_Urban,Unemployment,PopDen_Urban,PopPct_Rural,PopDen_Rural
0,0,13025,5947,484.0,19456.0,0.669459,0.305664,0.024877,Apache,AZ,2000,25.94,9.0,1080.7,74.06,4.7
1,0,13360,18180,1701.0,33241.0,0.401913,0.546915,0.051172,Cochise,AZ,2000,63.70,4.5,1933.4,36.30,7.8
2,0,20280,17562,3041.0,40883.0,0.496050,0.429567,0.074383,Coconino,AZ,2000,68.53,4.4,1939.8,31.47,2.3
3,0,7700,9158,878.0,17736.0,0.434145,0.516351,0.049504,Gila,AZ,2000,58.94,5.2,1625.0,41.06,4.6
4,0,3355,6007,302.0,9664.0,0.347165,0.621585,0.031250,Graham,AZ,2000,53.56,5.4,1430.2,46.44,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,0,79200,142521,17321.0,239042.0,0.331323,0.596217,0.072460,Waukesha,WI,2016,90.14,3.4,1314.5,9.86,136.2
2016,0,8440,16189,1395.0,26024.0,0.324316,0.622080,0.053604,Waupaca,WI,2016,35.06,3.9,1606.2,64.94,46.2
2017,0,3792,7669,538.0,11999.0,0.316026,0.639137,0.044837,Waushara,WI,2016,10.50,5.0,1403.6,89.50,35.1
2018,0,37054,43448,6642.0,87144.0,0.425204,0.498577,0.076219,Winnebago,WI,2016,86.57,3.6,2269.9,13.43,60.5


In [7]:
# Generate our categorical variable list
votes_cat = votes_df.dtypes[votes_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
votes_df[votes_cat].nunique()

county    355
state       6
dtype: int64

In [8]:
# creating instance of labelencoder
le = LabelEncoder()
# Assigning numerical values and storing in another column

votes_encoded_df = votes_df.apply(le.fit_transform)
votes_encoded_df

,index,blue_votes,red_votes,other_votes,total_votes,percent_blue,percent_red,percent_other,county,state,election_year,PopPct_Urban,Unemployment,PopDen_Urban,PopPct_Rural,PopDen_Rural
0,0,1115,471,398,813,1872,38,1329,12,0,0,71,67,75,283,11
1,0,1133,1138,824,1107,763,958,1773,74,0,0,230,22,283,124,20
2,0,1373,1120,965,1204,1415,262,1908,75,0,0,247,21,285,107,3
3,0,829,739,599,748,986,734,1748,119,0,0,211,29,227,143,10
4,0,318,479,260,378,433,1442,1462,124,0,0,190,31,168,164,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,0,1777,1867,1154,1768,368,1296,1904,339,5,4,316,11,130,38,333
2016,0,889,1085,763,994,335,1448,1791,340,5,4,115,16,220,239,168
2017,0,369,622,434,497,298,1538,1694,341,5,4,15,27,156,339,120
2018,0,1589,1570,1101,1541,914,611,1910,347,5,4,300,13,326,54,216


In [9]:
# Fit and transform the OneHotEncoder using the categorical variable list
#encode_df = pd.DataFrame(enc.fit_transform(nn_df[votes_mi_cat]))

# Add the encoded variable names to the DataFrame
#encode_df.columns = enc.get_feature_names(votes_mi_cat)
#encode_df.head()
X = votes_encoded_df
X.columns.to_list()
X = X.fillna(0)
X.isnull().sum().values.any()

False

In [10]:
# Create a y series from Transactions column
y = X["total_votes"]

In [11]:
# Split X and y into training and testing sets
X = X.drop("total_votes", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net
nn = tf.keras.models.Sequential()

In [14]:
# What is the length of the X_train info
len(X_train_scaled[0])

15

In [15]:
# First hidden layer
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
    )

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

In [17]:
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [18]:
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 128       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 179
Trainable params: 179
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model using the "adam" optimizer and "mean_squared_error" or "mean_absolute_error" loss function
#nn.compile(loss="mean_absolute_error", optimizer="adam", metrics=["accuracy"])
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model for 100 epochs.
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1515/1515 [==============================] - 0s 132us/sample - loss: 1329089.2208 - acc: 6.6007e-04
Epoch 2/100
1515/1515 [==============================] - 0s 41us/sample - loss: 1328931.0165 - acc: 6.6007e-04
Epoch 3/100
1515/1515 [==============================] - 0s 43us/sample - loss: 1328717.6278 - acc: 6.6007e-04
Epoch 4/100
1515/1515 [==============================] - 0s 42us/sample - loss: 1328500.2560 - acc: 6.6007e-04
Epoch 5/100
1515/1515 [==============================] - 0s 49us/sample - loss: 1328357.7912 - acc: 6.6007e-04
Epoch 6/100
1515/1515 [==============================] - 0s 47us/sample - loss: 1328289.9258 - acc: 6.6007e-04
Epoch 7/100
1515/1515 [==============================] - 0s 44us/sample - loss: 1328257.5672 - acc: 6.6007e-04
Epoch 8/100
1515/1515 [==============================] - 0s 43us/sample - loss: 1328241.2571 - acc: 6.6007e-04
Epoch 9/100
1515/1515 [==============================] - 0s 48us/sample - loss: 1328232.2300 - acc: 6.6007e-04


1515/1515 [==============================] - 0s 59us/sample - loss: 1328211.4294 - acc: 6.6007e-04
Epoch 75/100
1515/1515 [==============================] - 0s 51us/sample - loss: 1328211.4125 - acc: 6.6007e-04
Epoch 76/100
1515/1515 [==============================] - 0s 57us/sample - loss: 1328211.4063 - acc: 6.6007e-04
Epoch 77/100
1515/1515 [==============================] - 0s 63us/sample - loss: 1328211.3810 - acc: 6.6007e-04
Epoch 78/100
1515/1515 [==============================] - 0s 69us/sample - loss: 1328211.3932 - acc: 6.6007e-04
Epoch 79/100
1515/1515 [==============================] - 0s 52us/sample - loss: 1328211.3590 - acc: 6.6007e-04
Epoch 80/100
1515/1515 [==============================] - 0s 56us/sample - loss: 1328211.3861 - acc: 6.6007e-04
Epoch 81/100
1515/1515 [==============================] - 0s 59us/sample - loss: 1328211.3743 - acc: 6.6007e-04
Epoch 82/100
1515/1515 [==============================] - 0s 57us/sample - loss: 1328211.3659 - acc: 6.6007e-04
Epoch

In [21]:
# Show model info summary
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

505/505 - 0s - loss: 1288212.4738 - acc: 0.0000e+00
Loss: 1288212.4737623762, Accuracy: 0.0


In [22]:
# Predict values for the train and test sets
y_pred = nn.predict(X_test_scaled)
y_pred_train = nn.predict(X_train_scaled)
y_pred_train

array([[0.9999923 ],
       [0.999998  ],
       [0.9999987 ],
       ...,
       [1.        ],
       [1.        ],
       [0.99990416]], dtype=float32)

In [23]:
# Score the training predictions with r2_score()
r2_score(y_train, y_pred_train)

-2.9953039049308092